# What's New Section Updater

Written by: Jack A. Cole, Ph.D.
Mindware Consulting, Inc.
https://www.linkedin.com/in/jack-cole-b84b5638/


This notebook will update the "What's New" field in App Store Connect for the latest release of the app.  This is a strong pain point for apps that have many languages, as the App Store Connect UI does not allow for bulk editing of the "What's New" field.  This notebook will allow you to update the "What's New" field for all languages at once.

# Instructions

In App Store Connect, go to Users and Access/Integrations.

Create a new API key and download the key file.  You will need the key ID, issuer ID, and the key file.

Create a new app version for your app in App Store Connect.  You will need the app ID and the version ID.

Enter all of the above information in the cell below.

Enter the text for the "What's New" field in the cell below.  This will be automatically translated into all languages for your app.

Run the notebook.

In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install Pyjwt

In [ ]:
# Define your credentials
issuer_id = ''
key_id = ''
private_key_file_path = r''
# Define your app's ID and version ID
your_app_id = ""
your_version_id = ""

# Your app's new features in English
new_features_en = "This is a minor update, which includes bug fixes and performance improvements."


In [10]:
import requests
import json
from googletrans import Translator
import jwt
import time
from pathlib import Path

language_code_mapping = {
    'zh-Hans': 'zh-CN',  # Simplified Chinese
    'zh-Hant': 'zh-TW',  # Traditional Chinese
    'ar-SA': 'ar',       # Arabic (Saudi Arabia)
    'nl-NL': 'nl',       # Dutch (Netherlands)
    'en-US': 'en',       # English (United States)
    'en-GB': 'en',       # English (Great Britain)
    'en-CA': 'en',       # English (Canada)
    'en-AU': 'en',       # English (Australia)
    'fr-CA': 'fr',       # French (Canada)
    'pt-BR': 'pt',       # Portuguese (Brazil)
    'es-MX': 'es',       # Spanish (Mexico)
    'es-ES': 'es',       # Spanish (Spain)
    'pt-PT': 'pt',       # Portuguese
    'de-DE': 'de',       # German
    'fr-FR': 'fr',       # French    
    # Add other mappings as needed
    # For other languages not listed here, you can default to their two-letter ISO 639-1 codes
}

def create_jwt_token(issuer_id, key_id, private_key_file):
    # The token is valid for 20 minutes
    expiration_time = int(time.time()) + (20 * 60)

    # Read the private key
    with Path(private_key_file).open('r') as file:
        private_key = file.read()

    # Payload for the JWT
    payload = {
        'iss': issuer_id,
        'exp': expiration_time,
        'aud': 'appstoreconnect-v1'
    }

    # Header for the JWT
    header = {
        'alg': 'ES256',
        'kid': key_id,
        'typ': 'JWT'
    }

    # Generate the JWT token
    token = jwt.encode(payload, private_key, algorithm='ES256', headers=header)

    return token

# Function to get appStoreVersionId for a specific version
def get_app_store_version_id(app_id, version, auth_token):
    url = f"https://api.appstoreconnect.apple.com/v1/apps/{app_id}/appStoreVersions"
    headers = get_headers(auth_token)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        versions = response.json()['data']
        for version_info in versions:
            if version_info['attributes']['versionString'] == version:
                return version_info['id']
    else:
        print(f"Failed to retrieve app store versions: {response.text}")
        return None

# Function to get JWT token header
def get_headers(auth_token):
    return {
        'Authorization': f'Bearer {auth_token}',
        'Content-Type': 'application/json'
    }

# Function to list apps
def list_apps(auth_token):
    url = "https://api.appstoreconnect.apple.com/v1/apps"
    headers = get_headers(auth_token)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        apps = response.json()
        return apps['data']
    else:
        print(f"Failed to list apps: {response.text}")
        return []

# Function to get app_info_id
def get_app_info_id(app_id, auth_token):
    # First, get the related link for appInfos
    url = f"https://api.appstoreconnect.apple.com/v1/apps/{app_id}/relationships/appInfos"
    headers = get_headers(auth_token)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        related_link = response.json()['links']['related']
        # Then, make a request to the related link to get appInfos
        response = requests.get(related_link, headers=headers)
        if response.status_code == 200:
            app_infos = response.json()['data']
            if app_infos:
                return app_infos[0]['id']  # Assuming you want the first appInfo ID
            else:
                print("No appInfos found for this app.")
                return None
        else:
            print(f"Failed to retrieve app info from related link: {response.text}")
            return None
    else:
        print(f"Failed to retrieve related link for app infos: {response.text}")
        return None

# Function to get supported languages and their localization IDs for a specific app version
def get_supported_languages(app_store_version_id, auth_token):
    url = f"https://api.appstoreconnect.apple.com/v1/appStoreVersions/{app_store_version_id}/appStoreVersionLocalizations"
    headers = get_headers(auth_token)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        localizations = response.json()['data']
        languages_and_ids = {loc['attributes']['locale']: loc['id'] for loc in localizations}
        return languages_and_ids
    else:
        print(f"Failed to retrieve localizations for app version: {response.text}")
        return {}

def update_whats_new(localization_id, whats_new_text, auth_token):
    url = f"https://api.appstoreconnect.apple.com/v1/appStoreVersionLocalizations/{localization_id}"
    payload = json.dumps({
        'data': {
            'type': 'appStoreVersionLocalizations',
            'id': localization_id,
            'attributes': {
                'whatsNew': whats_new_text
            }
        }
    })
    response = requests.patch(url, headers=get_headers(auth_token=auth_token), data=payload)
    if response.status_code == 200:
        print(f"Successfully updated for localization ID: {localization_id}")
    else:
        print(f"Failed to update for localization ID {localization_id}: {response.text}")

def translate_and_back_translate(text, app_store_language_code):
    # Map App Store language code to Google Translate code
    google_translate_code = language_code_mapping.get(app_store_language_code, app_store_language_code)
    
    try:
        translator = Translator()
        # Translate to the target language
        translated_text = translator.translate(text, dest=google_translate_code).text
        # Back translate to English
        back_translated_text = translator.translate(translated_text, dest='en').text
        return translated_text, back_translated_text
    except Exception as e:
        print(f"Error in translation for {app_store_language_code} ({google_translate_code}): {e}")
        return None, None

# Main execution block
if __name__ == "__main__":

    auth_token = create_jwt_token(issuer_id, key_id, private_key_file_path)

    apps = list_apps(auth_token)
    if apps:
        app_id = apps[0]['id']  # or your specific app ID
        app_store_version_id = get_app_store_version_id(app_id, your_version_id, auth_token)
        if app_store_version_id:
            languages_and_ids = get_supported_languages(app_store_version_id, auth_token)
            for lang, localization_id in languages_and_ids.items():
                translation, back_translation = translate_and_back_translate(new_features_en, lang)
                if translation and back_translation:
                    print(f"Language: {lang}, Translation: {translation}, Back Translation: {back_translation}")
                    update_whats_new(localization_id, translation, auth_token)
                else:
                    print(f"Skipping update for language {lang} due to translation error.")
            print("Supported Languages:", list(languages_and_ids.keys()))
        else:
            print("Failed to find app store version ID.")
    else:
        print("No apps found.")


Language: sv, Translation: Detta är en mindre uppdatering, som inkluderar bugfixar och prestandaförbättringar., Back Translation: This is a minor update, which includes bug fixes and performance improvements.
Successfully updated for localization ID: 830d283d-f0ff-4621-a17d-ea0065f842f7
Language: it, Translation: Questo è un aggiornamento minore, che include correzioni di bug e miglioramenti delle prestazioni., Back Translation: This is a minor update, which includes bug corrections and performance improvements.
Successfully updated for localization ID: ac64de4e-6a6a-4cc9-a7b6-65a8ac8a7215
Language: sk, Translation: Toto je menšia aktualizácia, ktorá obsahuje opravy chýb a vylepšenia výkonu., Back Translation: This is a smaller update that contains mistakes and power improvements.
Successfully updated for localization ID: bc915ec3-66d5-4e13-808e-a1f6f97495cb
Language: zh-Hans, Translation: 这是一个较小的更新，其中包括错误修复和性能改进。, Back Translation: This is a smaller update, including error repair and 